# 🎨 ComfyUI Colab — Batch Workflow Runner

A clean, modular setup for running ComfyUI in Google Colab with a built-in batch processor.

**Run cells top to bottom.** Code is hidden by default — expand to edit.

---
| Step | What it does |
|------|-------------|
| 1 · Mount Drive | Connects Google Drive |
| 2 · Install | ComfyUI + speed stack (xformers, triton, sageattention) |
| 3 · Custom Nodes | Install nodes from GitHub URLs |
| 4 · Models | Download models via aria2c |
| 5 · Launch | Starts ComfyUI + batch watcher |


In [1]:
# @title 📂 1 · Mount Google Drive { display-mode: "form" }
# @markdown
from google.colab import drive
import os

print("📂 Connecting to Google Drive...")
# This will handle both MyDrive and Shared Drives
drive.mount('/content/drive', force_remount=True)

# Verify access
if os.path.exists("/content/drive/Shareddrives"):
    print("✅ Shared Drives detected.")
elif os.path.exists("/content/drive/Shared drives"):
     print("✅ Shared Drives detected (alternate path).")
else:
    print("ℹ️ Only MyDrive detected.")

print("✅ Drive mount complete.")


📂 Connecting to Google Drive...
Mounted at /content/drive
✅ Shared Drives detected.
✅ Drive mount complete.


In [2]:
# @title ⚙️ Install ComfyUI & Dependencies { display-mode: "form" }
# @markdown ## 2 · Install ComfyUI & Dependencies
# @markdown
# @markdown Clones ComfyUI and installs the full stack including the **speed trio**:
# @markdown - **xformers** — faster attention, lower VRAM
# @markdown - **triton** — GPU kernel acceleration
# @markdown - **sageattention** — additional attention optimization
# @markdown
# @markdown Also sets `PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True` to reduce OOM crashes.
import os
from pathlib import Path

WORKSPACE = "/content/ComfyUI"

# ── 0. Install uv ─────────────────────────────────────────────────
print("📦 Installing uv...")
!curl -LsSf https://astral.sh/uv/install.sh | sh
os.environ["PATH"] = "/root/.local/bin:" + os.environ["PATH"]
print("✅ uv ready\n")

# ── 1. Clone / update ComfyUI ─────────────────────────────────────
if not os.path.exists(WORKSPACE):
    print("📥 Cloning ComfyUI...")
    !git clone -q https://github.com/comfyanonymous/ComfyUI {WORKSPACE}
    print("✅ Cloned\n")
else:
    print("✅ ComfyUI exists, pulling updates...")
    !cd {WORKSPACE} && git pull -q
    print("✅ Updated\n")

%cd {WORKSPACE}

# ── 2. PyTorch ────────────────────────────────────────────────────
print("⚡ Installing PyTorch 2.8.0...")
!uv pip install --system torch==2.8.0 torchvision==0.23.0 torchaudio==2.8.0 --no-deps

# ── 3. Speed stack ────────────────────────────────────────────────
print("🚀 Installing speed stack...")
!uv pip install --system xformers==0.0.32.post1 triton==3.4 sageattention==1.0.6

# ── 4. ComfyUI requirements ───────────────────────────────────────
print("📦 Installing ComfyUI requirements...")
!uv pip install --system -r requirements.txt

# ── 5. Core dependencies ──────────────────────────────────────────
print("📚 Installing core dependencies...")
!uv pip install --system \
    accelerate einops diffusers \
    "safetensors>=0.4.2" \
    aiohttp pyyaml Pillow scipy tqdm psutil \
    "tokenizers>=0.13.3" sentencepiece soundfile \
    "kornia>=0.7.1" spandrel torchsde \
    av albumentations opencv-python \
    onnxruntime-gpu color-matcher \
    comfy_aimdo comfy-kitchen

# ── 6. Transformers / HuggingFace ────────────────────────────────
print("🤗 Installing transformers & huggingface-hub...")
!uv pip install --system \
    "transformers>=4.45.0,<5.0.0" \
    "huggingface-hub>=0.23.0" \
    hf_transfer

# ── 7. CUDA memory optimization ──────────────────────────────────
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
print("✅ CUDA memory config set\n")

# ── 8. ComfyUI Manager ────────────────────────────────────────────
manager_path = f"{WORKSPACE}/custom_nodes/ComfyUI-Manager"
if not os.path.exists(manager_path):
    print("📥 Installing ComfyUI Manager...")
    !git clone -q https://github.com/ltdrdata/ComfyUI-Manager {manager_path}
else:
    print("🔄 Updating ComfyUI Manager...")
    !cd {manager_path} && git pull -q
print("✅ ComfyUI Manager ready\n")

# ── 9. Verify ─────────────────────────────────────────────────────
import importlib.metadata as meta
print("📋 Key package versions:")
for pkg in ["torch", "xformers", "transformers", "huggingface-hub", "safetensors"]:
    try:
        print(f"  ✅ {pkg}: {meta.version(pkg)}")
    except Exception:
        print(f"  ❌ {pkg}: not found")

print("\n🎉 Installation complete! Run the next cell.")


📦 Installing uv...
downloading uv 0.10.2 x86_64-unknown-linux-gnu
no checksums to verify
installing to /usr/local/bin
  uv
  uvx
everything's installed!
✅ uv ready

📥 Cloning ComfyUI...
✅ Cloned

/content/ComfyUI
⚡ Installing PyTorch 2.8.0...
Using Python 3.12.12 environment at: /usr
Resolved 3 packages in 26ms
Prepared 3 packages in 11.23s
Uninstalled 3 packages in 839ms
Installed 3 packages in 220ms
 - torch==2.9.0+cu128
 + torch==2.8.0
 - torchaudio==2.9.0+cu128
 + torchaudio==2.8.0
 - torchvision==0.24.0+cu128
 + torchvision==0.23.0
🚀 Installing speed stack...
Using Python 3.12.12 environment at: /usr
Resolved 28 packages in 92ms
Prepared 4 packages in 6.13s
Uninstalled 2 packages in 36ms
Installed 4 packages in 11ms
 - nvidia-nccl-cu12==2.27.5
 + nvidia-nccl-cu12==2.27.3
 + sageattention==1.0.6
 - triton==3.5.0
 + triton==3.4.0
 + xformers==0.0.32.post1
📦 Installing ComfyUI requirements...
Using Python 3.12.12 environment at: /usr
Resolved 88 packages in 689ms
Prepared 16 packages

In [3]:
# @title 🔧 Custom Nodes { display-mode: "form" }
# @markdown ## 3 · Custom Nodes
# @markdown
# @markdown Add GitHub repo URLs to `CUSTOM_NODES` as `("folder_name", "url", "extra_flags")`.
# @markdown Extra flags are optional — e.g. `"--branch main"` or `""` for none.
import os

WORKSPACE        = "/content/ComfyUI"
CUSTOM_NODES_DIR = f"{WORKSPACE}/custom_nodes"

# ╔══════════════════════════════════════════════════════════════════╗
# ║  ADD YOUR NODES                                                 ║
# ╚══════════════════════════════════════════════════════════════════╝

CUSTOM_NODES = [
    ("ComfyUI_GGUF",                          "https://github.com/Isi-dev/ComfyUI_GGUF.git",                   "--branch forQwen"),
    ("ComfyUI_DeleteModelPassthrough",         "https://github.com/Isi-dev/ComfyUI_DeleteModelPassthrough.git", ""),
    ("comfyui_controlnet_aux",                 "https://github.com/Isi-dev/comfyui_controlnet_aux",             ""),
    ("ComfyUI-WanVideoWrapper",                "https://github.com/kijai/ComfyUI-WanVideoWrapper",              ""),
    ("ComfyUI-VideoHelperSuite",               "https://github.com/Kosinkadink/ComfyUI-VideoHelperSuite",       ""),
    ("ComfyUI-KJNodes",                        "https://github.com/kijai/ComfyUI-KJNodes.git",                  ""),
    ("ComfyUI-segment-anything-2",             "https://github.com/kijai/ComfyUI-segment-anything-2",           ""),
    ("ComfyUI-Florence2",                      "https://github.com/kijai/ComfyUI-Florence2",                    ""),
    ("ComfyUI-Inspyrenet-Rembg",               "https://github.com/john-mnz/ComfyUI-Inspyrenet-Rembg.git",     ""),
    ("ComfyUI_Animation_Nodes_and_Workflows",  "https://github.com/Isi-dev/ComfyUI_Animation_Nodes_and_Workflows", ""),
]

# ╔══════════════════════════════════════════════════════════════════╗

if not CUSTOM_NODES:
    print("ℹ️  No custom nodes configured.")
else:
    print("🔧 Installing custom nodes...\n")
    for name, url, flags in CUSTOM_NODES:
        path = f"{CUSTOM_NODES_DIR}/{name}"
        if not os.path.exists(path):
            print(f"  📥 {name}...")
            os.system(f"git clone -q {flags} {url} {path}")
            print(f"  ✅ {name}")
        else:
            print(f"  ⏭️  {name} (exists)")

    print("\n📚 Installing node requirements...")
    for name, _, _ in CUSTOM_NODES:
        req = f"{CUSTOM_NODES_DIR}/{name}/requirements.txt"
        if os.path.exists(req):
            print(f"  📦 {name}...")
            os.system(f"uv pip install --system -r {req} -q")
    print("\n✅ Custom nodes ready! Run the next cell.")


🔧 Installing custom nodes...

  📥 ComfyUI_GGUF...
  ✅ ComfyUI_GGUF
  📥 ComfyUI_DeleteModelPassthrough...
  ✅ ComfyUI_DeleteModelPassthrough
  📥 comfyui_controlnet_aux...
  ✅ comfyui_controlnet_aux
  📥 ComfyUI-WanVideoWrapper...
  ✅ ComfyUI-WanVideoWrapper
  📥 ComfyUI-VideoHelperSuite...
  ✅ ComfyUI-VideoHelperSuite
  📥 ComfyUI-KJNodes...
  ✅ ComfyUI-KJNodes
  📥 ComfyUI-segment-anything-2...
  ✅ ComfyUI-segment-anything-2
  📥 ComfyUI-Florence2...
  ✅ ComfyUI-Florence2
  📥 ComfyUI-Inspyrenet-Rembg...
  ✅ ComfyUI-Inspyrenet-Rembg
  📥 ComfyUI_Animation_Nodes_and_Workflows...
  ✅ ComfyUI_Animation_Nodes_and_Workflows

📚 Installing node requirements...
  📦 ComfyUI_GGUF...
  📦 ComfyUI_DeleteModelPassthrough...
  📦 comfyui_controlnet_aux...
  📦 ComfyUI-WanVideoWrapper...
  📦 ComfyUI-VideoHelperSuite...
  📦 ComfyUI-KJNodes...
  📦 ComfyUI-Florence2...
  📦 ComfyUI-Inspyrenet-Rembg...
  📦 ComfyUI_Animation_Nodes_and_Workflows...

✅ Custom nodes ready! Run the next cell.


In [10]:
# @title 📥 Models { display-mode: "form" }
# @markdown ## 4 · Models
# @markdown
# @markdown Add download URLs to `MODELS` as `("url", "dest_subfolder")`.
# @markdown Dest subfolder is relative to `/content/ComfyUI/models/` — e.g. `"checkpoints"`, `"loras"`, `"vae"`.
# @markdown Civitai URLs supported — add your token to `CIVITAI_TOKEN` if needed.
import os, subprocess
from pathlib import Path

WORKSPACE  = "/content/ComfyUI"
MODELS_DIR = f"{WORKSPACE}/models"

CIVITAI_TOKEN = ""  # @param {type:"string"}

# ╔══════════════════════════════════════════════════════════════════╗
# ║  ADD YOUR MODELS                                                ║
# ╚══════════════════════════════════════════════════════════════════╝

MODELS = [
    # WanVideo & SAM2 (Existing)
    ("https://huggingface.co/Kijai/WanVideo_comfy_fp8_scaled/resolve/main/Wan22Animate/Wan2_2-Animate-14B_fp8_e4m3fn_scaled_KJ.safetensors", "diffusion_models"),
    ("https://huggingface.co/Comfy-Org/Wan_2.2_ComfyUI_Repackaged/resolve/main/split_files/text_encoders/umt5_xxl_fp8_e4m3fn_scaled.safetensors", "text_encoders"),
    ("https://huggingface.co/Comfy-Org/Wan_2.1_ComfyUI_repackaged/resolve/main/split_files/vae/wan_2.1_vae.safetensors", "vae"),
    ("https://huggingface.co/Isi99999/Wan_Extras/resolve/main/clip_vision_h.safetensors", "clip_vision"),
    ("https://huggingface.co/Kijai/sam2-safetensors/resolve/main/sam2.1_hiera_small.safetensors", "sam2"),
    ("https://huggingface.co/Kijai/WanVideo_comfy/resolve/main/Lightx2v/lightx2v_T2V_14B_cfg_step_distill_v2_lora_rank32_bf16.safetensors", "loras"),
    ("https://huggingface.co/Comfy-Org/Wan_2.2_ComfyUI_Repackaged/resolve/main/split_files/loras/wan2.2_animate_14B_relight_lora_bf16.safetensors", "loras"),
]

# ╔══════════════════════════════════════════════════════════════════╗

def ensure_aria2():
    try:
        subprocess.run(["aria2c", "--version"], capture_output=True, check=True)
    except (FileNotFoundError, subprocess.CalledProcessError):
        subprocess.run(["apt-get", "-y", "install", "-qq", "aria2"],
                       check=True, capture_output=True)

def download(url, dest_dir):
    Path(dest_dir).mkdir(parents=True, exist_ok=True)
    filename = url.split("/")[-1].split("?")[0]
    dest_path = os.path.join(dest_dir, filename)

    # Check if file already exists
    if os.path.exists(dest_path):
        print(f"  ⏭️  {filename} (exists)")
        return filename

    print(f"  ⬇️  Downloading {filename}...")

    if "civitai.com" in url.lower():
        import time
        model_id = url.split("/models/")[1].split("?")[0]
        api_url  = f"https://civitai.com/api/download/models/{model_id}?type=Model&format=SafeTensor"
        if CIVITAI_TOKEN:
            api_url += f"&token={CIVITAI_TOKEN}"
        filename = f"civitai_{model_id}_{time.strftime('%H%M%S')}.safetensors"
        dest_path = os.path.join(dest_dir, filename)
        # wget already shows progress bar by default with --show-progress
        os.system(f'wget --max-redirect=10 --show-progress "{api_url}" -O "{dest_path}"')
    else:
        # CHANGED: Switched to os.system and removed the quiet flags.
        # This will show the [###...] progress bar and speed in the output.
        cmd = f'aria2c -c -x 16 -s 16 -k 1M -d "{dest_dir}" -o "{filename}" "{url}"'
        os.system(cmd)

    return filename

if not MODELS:
    print("ℹ️  No models configured.")
else:
    ensure_aria2()
    print("📥 Downloading models...\n")
    for url, subfolder in MODELS:
        download(url, f"{MODELS_DIR}/{subfolder}")
    print("\n✅ Models ready! Run the Launch cell next.")


📥 Downloading models...

  ⏭️  Wan2_2-Animate-14B_fp8_e4m3fn_scaled_KJ.safetensors (exists)
  ⏭️  umt5_xxl_fp8_e4m3fn_scaled.safetensors (exists)
  ⏭️  wan_2.1_vae.safetensors (exists)
  ⏭️  clip_vision_h.safetensors (exists)
  ⏭️  sam2.1_hiera_small.safetensors (exists)
  ⏭️  lightx2v_T2V_14B_cfg_step_distill_v2_lora_rank32_bf16.safetensors (exists)
  ⏭️  wan2.2_animate_14B_relight_lora_bf16.safetensors (exists)
  ⏭️  config.json (exists)
  ⏭️  model.safetensors (exists)
  ⏭️  tokenizer.json (exists)
  ⏭️  tokenizer_config.json (exists)
  ⬇️  Downloading special_tokens_map.json...
  ⬇️  Downloading vocab.json...
  ⬇️  Downloading merges.txt...
  ⬇️  Downloading preprocessor_config.json...
  ⬇️  Downloading generation_config.json...

✅ Models ready! Run the Launch cell next.


In [12]:
# Florence-2 Base — must use snapshot_download, not individual files
from huggingface_hub import snapshot_download
snapshot_download(
    repo_id="microsoft/Florence-2-base",
    local_dir="/content/ComfyUI/models/LLM/Florence-2-base",
    ignore_patterns=["*.md", "*.txt", "*.gitattributes"]
)
print("✅ Florence-2-base downloaded")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

modeling_florence2.py: 0.00B [00:00, ?B/s]

preprocessor_config.json:   0%|          | 0.00/806 [00:00<?, ?B/s]

LICENSE: 0.00B [00:00, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/464M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

tokenizer_config.json:   0%|          | 0.00/34.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

✅ Florence-2-base downloaded


In [ ]:
# @title 🚀 Launch ComfyUI (Triplet Batch Mode) { display-mode: "form" }
import os, time, json, shutil, threading, subprocess, requests, uuid
from pathlib import Path
from datetime import datetime
from google.colab import drive, output

# ╔══════════════════════════════════════════════════════════════════╗
# ║  ⚙️  CONFIGURATION                                              ║
# ╚══════════════════════════════════════════════════════════════════╝

# @markdown ### 📂 Drive Paths
drive_subfolder   = "Figuro/character-swap"              # @param {type:"string"}
workflow_filename = "Wan_2_2_Animate_workflow_api.json"  # @param {type:"string"}

# @markdown ### 📤 Extra Output (Optional)
extra_output_path = "Figuro/video-upscale/input"         # @param {type:"string"}

# @markdown ### 🌐 Launch Mode
launch_mode    = "window"  # @param ["window", "iframe", "cloudflare"]
check_interval = 10        # @param {type:"slider", min:5, max:60, step:5}

# @markdown ### 🤖 Workflow Node IDs
node_video = "63"
node_image = "57"
node_text  = "178"
output_node_id = "182"  # @param {type:"string"}

# ╔══════════════════════════════════════════════════════════════════╗
# ║  🔍  PATH RESOLUTION                                            ║
# ╚══════════════════════════════════════════════════════════════════╝

def find_drive_base(subfolder: str) -> str:
    roots = ["/content/drive/Shareddrives", "/content/drive/Shared drives", "/content/drive/MyDrive"]
    for r in roots:
        candidate = os.path.join(r, subfolder)
        if os.path.isdir(candidate):
            return candidate
    try:
        for top in os.listdir("/content/drive"):
            candidate = os.path.join("/content/drive", top, subfolder)
            if os.path.isdir(candidate):
                return candidate
    except:
        pass
    raise RuntimeError(f"❌ Path not found: {subfolder}")

if not os.path.exists("/content/drive"):
    drive.mount('/content/drive')

DRIVE_BASE = find_drive_base(drive_subfolder)
folders = {
    "input":     f"{DRIVE_BASE}/input",
    "processed": f"{DRIVE_BASE}/processed",
    "output":    f"{DRIVE_BASE}/output",
}
for p in folders.values():
    os.makedirs(p, exist_ok=True)

EXTRA_DIR = None
if extra_output_path.strip():
    try:
        EXTRA_DIR = find_drive_base(extra_output_path.strip())
    except:
        EXTRA_DIR = f"/content/drive/MyDrive/{extra_output_path.strip()}"
        os.makedirs(EXTRA_DIR, exist_ok=True)

WF_PATH = os.path.join(DRIVE_BASE, workflow_filename)
print(f"✅ Paths ready.")
print(f"   Watching : {folders['input']}")
print(f"   Workflow : {WF_PATH}")

# ╔══════════════════════════════════════════════════════════════════╗
# ║  🔧  COMFYUI HELPERS                                            ║
# ╚══════════════════════════════════════════════════════════════════╝

COMFYUI_URL = "http://127.0.0.1:8188"
_ready_event = threading.Event()  # fired exactly once when ComfyUI is up

def wait_for_ready():
    """Block until ComfyUI responds, then fire _ready_event exactly once."""
    while not _ready_event.is_set():
        try:
            if requests.get(f"{COMFYUI_URL}/system_stats", timeout=2).status_code == 200:
                time.sleep(2)  # small buffer so custom nodes fully load
                _ready_event.set()
                print("\n[Batch Runner] ✅ COMFYUI IS READY!")
                return
        except:
            pass
        time.sleep(2)

def process_triplet(stem, group):
    print(f"\n[Batch Runner] ⭐ STARTING: {stem}")
    comfy_input = "/content/ComfyUI/input"
    os.makedirs(comfy_input, exist_ok=True)
    shutil.copy2(group['video'], os.path.join(comfy_input, Path(group['video']).name))
    shutil.copy2(group['image'], os.path.join(comfy_input, Path(group['image']).name))

    with open(WF_PATH, 'r') as f:
        wf = json.load(f)
    with open(group['text'], 'r', encoding='utf-8') as f:
        prompt_text = f.read().strip()

    wf[node_video]["inputs"]["video"] = Path(group['video']).name
    wf[node_image]["inputs"]["image"] = Path(group['image']).name
    wf[node_text]["inputs"]["text"]   = prompt_text

    p = {"prompt": wf, "client_id": str(uuid.uuid4())}
    r = requests.post(f"{COMFYUI_URL}/prompt", json=p, timeout=30).json()
    if "error" in r:
        raise RuntimeError(f"ComfyUI rejected prompt: {r['error']}")
    prompt_id = r["prompt_id"]
    print(f"[Batch Runner] 📋 Prompt ID: {prompt_id}")

    # Poll until done (30 min max)
    deadline = time.time() + 1800
    h = {}
    while time.time() < deadline:
        time.sleep(5)
        try:
            h = requests.get(f"{COMFYUI_URL}/history/{prompt_id}", timeout=10).json()
            if prompt_id in h:
                break
        except:
            pass
    else:
        raise TimeoutError(f"Prompt {prompt_id} did not finish within 30 min")

    # --- RELIABLE LOCAL DISK COPY LOGIC ---
    outputs = h[prompt_id].get("outputs", {})
    saved = []

    if output_node_id in outputs:
        node_data = outputs[output_node_id]
        # Check all possible media keys
        items = node_data.get("videos") or node_data.get("gifs") or node_data.get("images") or []

        for item in items:
            file_name = item['filename']
            sub_folder = item.get('subfolder', '')

            # The actual path on the local Colab Disk
            local_path = os.path.join("/content/ComfyUI/output", sub_folder, file_name)

            if os.path.exists(local_path):
                ts = datetime.now().strftime('%H%M%S')
                drive_out_name = f"{stem}_{ts}{Path(file_name).suffix}"
                drive_path = os.path.join(folders["output"], drive_out_name)

                # Copy from Local Disk to Google Drive
                shutil.copy2(local_path, drive_path)
                print(f"[Batch Runner] 💾 Saved to Drive: {drive_out_name}")
                saved.append(drive_path)

                if EXTRA_DIR:
                    shutil.copy2(drive_path, os.path.join(EXTRA_DIR, drive_out_name))
            else:
                print(f"[Batch Runner] ❌ Local file not found: {local_path}")
    else:
        print(f"[Batch Runner] ⚠️ Node {output_node_id} didn't produce a saved file. Check 'save_output' setting.")

    if not saved:
        print("[Batch Runner] ⚠️ No output files saved — check target node ID and workflow settings.")

    for fp in group['all']:
        dest = os.path.join(
            folders["processed"],
            f"{datetime.now().strftime('%Y%m%d_%H%M%S')}_{Path(fp).name}"
        )
        shutil.move(fp, dest)
    print(f"[Batch Runner] ✅ DONE: {stem}\n")

def watcher_loop():
    """Wait for ComfyUI then start polling the input folder."""
    _ready_event.wait()  # blocks until fired by wait_for_ready()
    print(f"[Batch Runner] 👀 Watcher active. Polling every {check_interval}s...")
    print(f"   Drop  <stem>.mp4 + <stem>.jpg + <stem>.txt  into input/ to trigger.\n")

    seen_errors = {}

    while True:
        try:
            files = os.listdir(folders["input"])
            groups = {}
            for f in files:
                if f.startswith("."):
                    continue
                path = os.path.join(folders["input"], f)
                stem = Path(f).stem
                ext  = Path(f).suffix.lower().lstrip(".")
                if stem not in groups:
                    groups[stem] = {"video": None, "image": None, "text": None, "all": []}
                groups[stem]["all"].append(path)
                if ext in {'mp4', 'mov', 'webm', 'mkv'}:
                    groups[stem]['video'] = path
                elif ext in {'jpg', 'jpeg', 'png', 'webp'}:
                    groups[stem]['image'] = path
                elif ext == 'txt':
                    groups[stem]['text'] = path

            for stem, g in groups.items():
                if seen_errors.get(stem, 0) >= 3:
                    continue
                if g['video'] and g['image'] and g['text']:
                    try:
                        process_triplet(stem, g)
                        seen_errors.pop(stem, None)
                    except Exception as e:
                        seen_errors[stem] = seen_errors.get(stem, 0) + 1
                        count = seen_errors[stem]
                        print(f"[Batch Runner] ❌ Error ({count}/3) '{stem}': {e}")
                        if count >= 3:
                            print(f"[Batch Runner] ⛔ Skipping '{stem}'. Remove files to retry.")
                elif g['video'] and g['image'] and not g['text']:
                    print(f"[Batch Runner] ⏳ '{stem}': waiting for .txt prompt file...")
                elif g['video'] and not g['image']:
                    print(f"[Batch Runner] ⏳ '{stem}': waiting for image...")
                elif g['image'] and not g['video']:
                    print(f"[Batch Runner] ⏳ '{stem}': waiting for video...")

        except Exception as e:
            print(f"[Batch Runner] ⚠️  Watcher error: {e}")

        time.sleep(check_interval)

# ╔══════════════════════════════════════════════════════════════════╗
# ║  🚀  LAUNCH                                                     ║
# ╚══════════════════════════════════════════════════════════════════╝

os.chdir("/content/ComfyUI")
print("\n--- STARTING COMFYUI SERVER ---")

# ── Launch ComfyUI with filtered log streaming ────────────────────
process = subprocess.Popen(
    ["python", "main.py", "--dont-print-server"],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1,
)

# Only print lines that are actually useful — skip the noise
_SKIP = {
    "DWPose:", "tensor.shape:", "WARNING: You need pytorch",
    "Found comfy_kitchen", "Using async weight", "Enabled pinned",
    "Checkpoint files will", "pytorch version:", "xformers version:",
    "Set vram state", "Device:", "Python version:", "ComfyUI version:",
    "ComfyUI frontend", "Prompt Server", "Context impl", "Will assume",
    "Assets scan", "[START] Security", "[DONE] Security",
    "installing dependencies", "startup time:", "Platform:", "Python executable:",
    "ComfyUI Path:", "Base Folder", "User directory:", "config path:",
    "Log path:", "Prestartup times", "Import times", "NumExpr",
    "tensorflow", "oneDNN", "Unable to register", "absl::", "E0000",
    "W0000", "computation placer", "TensorFlow binary",
    "AVX2", "ComfyUI-Manager] network", "ComfyUI-Manager] ComfyUI per-queue",
    "ComfyUI-Manager] default cache", "per-queue preview",
}

def stream_logs():
    for line in process.stdout:
        line = line.strip()
        if not line:
            continue
        if any(skip in line for skip in _SKIP):
            continue
        print(line, flush=True)

threading.Thread(target=stream_logs, daemon=True).start()

# ── Single ready-check thread ─────────────────────────────────────
threading.Thread(target=wait_for_ready, daemon=True).start()

# ── UI thread (fires only after _ready_event) ─────────────────────
def launch_ui():
    _ready_event.wait()
    if launch_mode == "iframe":
        output.serve_kernel_port_as_iframe(8188, height=900)
    elif launch_mode == "cloudflare":
        subprocess.Popen(
            ["cloudflared", "tunnel", "--url", "http://127.0.0.1:8188"],
            stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL
        )
    else:
        output.serve_kernel_port_as_window(8188)

threading.Thread(target=launch_ui, daemon=True).start()

# ── Watcher runs in main thread (keeps cell alive) ────────────────
try:
    watcher_loop()
except KeyboardInterrupt:
    print("\n🛑 Stopped by user.")
    process.terminate()

✅ Paths ready.
   Watching : /content/drive/Shareddrives/Figuro/character-swap/input
   Workflow : /content/drive/Shareddrives/Figuro/character-swap/Wan_2_2_Animate_workflow_api.json

--- STARTING COMFYUI SERVER ---
5.9 seconds: /content/ComfyUI/custom_nodes/ComfyUI-Manager
Total VRAM 22563 MB, total RAM 54229 MB
Using xformers attention
ComfyUI-GGUF: Allowing full torch compile
[/content/ComfyUI/custom_nodes/comfyui_controlnet_aux] | INFO -> Using ckpts path: /content/ComfyUI/custom_nodes/comfyui_controlnet_aux/ckpts
[/content/ComfyUI/custom_nodes/comfyui_controlnet_aux] | INFO -> Using symlinks: False
[/content/ComfyUI/custom_nodes/comfyui_controlnet_aux] | INFO -> Using ort providers: ['CUDAExecutionProvider', 'DirectMLExecutionProvider', 'OpenVINOExecutionProvider', 'ROCMExecutionProvider', 'CPUExecutionProvider', 'CoreMLExecutionProvider']
### Loading: ComfyUI-Manager (V3.39.2)
### ComfyUI Version: v0.13.0-34-g88e63705 | Released on '2026-02-15'
FETCH ComfyRegistry Data: 5/125
FET

<IPython.core.display.Javascript object>

FETCH ComfyRegistry Data: 20/125
FETCH ComfyRegistry Data: 25/125
FETCH ComfyRegistry Data: 30/125
FETCH ComfyRegistry Data: 35/125
FETCH ComfyRegistry Data: 40/125
FETCH ComfyRegistry Data: 45/125
FETCH ComfyRegistry Data: 50/125
FETCH ComfyRegistry Data: 55/125
FETCH ComfyRegistry Data: 60/125
FETCH ComfyRegistry Data: 65/125
FETCH ComfyRegistry Data: 70/125
FETCH ComfyRegistry Data: 75/125
FETCH ComfyRegistry Data: 80/125
FETCH ComfyRegistry Data: 85/125
FETCH ComfyRegistry Data: 90/125
FETCH ComfyRegistry Data: 95/125
FETCH ComfyRegistry Data: 100/125
FETCH ComfyRegistry Data: 105/125
FETCH ComfyRegistry Data: 110/125
FETCH ComfyRegistry Data: 115/125
FETCH ComfyRegistry Data: 120/125
FETCH ComfyRegistry Data: 125/125
FETCH ComfyRegistry Data [DONE]
FETCH DATA from: https://raw.githubusercontent.com/ltdrdata/ComfyUI-Manager/main/custom-node-list.json [DONE]
[ComfyUI-Manager] All startup tasks have been completed.
[Batch Runner] ⏳ '20260216_101250_test2': waiting for image...
[Batch

---

## 📝 Notes

**Workflow format** — Export via ComfyUI → Settings → Dev Mode → *Save (API Format)*. Uses numeric node IDs as top-level keys.

**Input node ID** — open `workflow.json`, find your Load Image / Load Video node, its top-level key (e.g. `"12"`) is `INPUT_NODE_ID`. The input field is auto-detected and printed on startup.

**Batch mode off** — set `BATCH_MODE = False` to just launch ComfyUI without the watcher. Useful when you want to use the UI manually.

**Launch modes** — `window` opens a new tab, `iframe` embeds in the cell, `cloudflare` gives a public shareable URL.

**Error handling** — files that fail 3 times are skipped permanently so the watcher never loops.

**Live edits** — workflow JSON is re-read on every file so you can tweak it without restarting.

---
